In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import warnings
import string
import nltk
from nltk.corpus import stopwords
from tensorflow import keras
import gensim.models
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from tensorboard.compat import tf2 as tf
import tensorflow as tf
from tensorflow.keras import initializers, preprocessing
import tensorflow_hub as hub
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, f1_score, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, plot_roc_curve
from sklearn.manifold import TSNE

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import enchant
import gensim

C:\Users\PC\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_TextOfImage = np.load('train_embeddings.npy')
val_TextOfImage = np.load('val_embeddings.npy')
test_TextOfImage = np.load('test_embeddings.npy')
train_TextOfImage = pd.DataFrame({'train_TextOfImage': train_TextOfImage})
val_TextOfImage = pd.DataFrame({'val_TextOfImage': val_TextOfImage})
test_TextOfImage = pd.DataFrame({'test_TextOfImage': test_TextOfImage})

In [6]:
# Définir un seuil de longueur minimale
seuil_longueur_minimale = 5

# Filtrer les lignes avec une longueur de texte inférieure au seuil
train_TextOfImage = train_TextOfImage[train_TextOfImage['train_TextOfImage'].apply(lambda x: len(str(x)) > seuil_longueur_minimale)]
val_TextOfImage = val_TextOfImage[val_TextOfImage['val_TextOfImage'].apply(lambda x: len(str(x)) > seuil_longueur_minimale)]
test_TextOfImage = test_TextOfImage[test_TextOfImage['test_TextOfImage'].apply(lambda x: len(str(x)) > seuil_longueur_minimale)]


In [8]:
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
import re
import string
from googletrans import Translator

# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
english_words = set(words.words())
translator = Translator()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [16]:
from langdetect import detect
from googletrans import Translator

def detect_and_translate_to_english(text):
    try:
        # Détection de la langue
        detected_lang = detect(text)
        
        # Traduction en anglais si la langue détectée n'est pas l'anglais
        if detected_lang != 'en':
            translator = Translator()
            translated_text = translator.translate(text, dest='en').text
            return translated_text
        else:
            return text
    except Exception as e:
        
        return text

# Appliquer la fonction à chaque élément de la colonne
train_TextOfImage['train_TextOfImage'] = train_TextOfImage['train_TextOfImage'].astype(str).apply(detect_and_translate_to_english)
val_TextOfImage['val_TextOfImage'] = val_TextOfImage['val_TextOfImage'].astype(str).apply(detect_and_translate_to_english)
test_TextOfImage['test_TextOfImage'] = test_TextOfImage['test_TextOfImage'].astype(str).apply(detect_and_translate_to_english)




In [29]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove numbers and punctuation
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Replace email addresses with the domain name
    email_regex = r'\S+@\S+'
    email_domains = [re.findall(r'@([\w.-]+)', email)[0] for email in re.findall(email_regex, text)]
    for email_domain in email_domains:
        text = re.sub(email_regex, email_domain, text)


    # Remove stopwords and words that are not in English
    words = text.split()
    words = [word for word in words if word not in stop_words and word in english_words]
    text = ' '.join(words)

    return text

# Data cleaning
train_TextOfImage['train_TextOfImage'] = train_TextOfImage['train_TextOfImage'].apply(clean_text)
val_TextOfImage['val_TextOfImage'] = val_TextOfImage['val_TextOfImage'].apply(clean_text)
test_TextOfImage['test_TextOfImage'] = test_TextOfImage['test_TextOfImage'].apply(clean_text)

In [30]:
# Supprimer les lignes avec la valeur 'none' dans la colonne 'test_TextOfImage'
test_TextOfImage = test_TextOfImage[test_TextOfImage['test_TextOfImage'] != 'none']

# Afficher les premières lignes après la suppression
print(test_TextOfImage.head())


Empty DataFrame
Columns: [test_TextOfImage]
Index: []


In [ ]:
# Sauvegarder le DataFrame train_TextOfImage
train_TextOfImage.to_csv('train_TextOfImage.csv', index=False)

# Sauvegarder le DataFrame val_TextOfImage
val_TextOfImage.to_csv('val_TextOfImage.csv', index=False)

# Sauvegarder le DataFrame test_TextOfImage
test_TextOfImage.to_csv('test_TextOfImage.csv', index=False)


In [2]:
data = pd.read_csv('EnronDatasetProjectNew.csv')
#data = pd.read_csv('bbc-text.csv')

In [8]:
unique_categories = data["category"].unique()
print(unique_categories)

['business' 'politics' 'sport' 'entertainment' 'tech']


In [6]:
data = data.dropna(subset=['category'])

In [7]:
data.shape

(6110, 5)

In [13]:
# Sélectionnez un index spécifique ou utilisez un index aléatoire
index = 70

# Récupérez le texte de l'e-mail à partir de la colonne "emailclean"
email_text = data['emailclean'][index]

# Affichez le texte de l'e-mail
print(email_text)

ami corp royal b ami subject wekend either forward ken lay se atached file mar mar ami corp royal b ami subject wekend either forward ken lay se atached file mar mar


In [126]:
# Initialiser les outils NLTK et PyEnchant
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [127]:
abbreviations = {
    "a/c": "à partir de",
    "adj.": "adjectif",
    "adv.": "adverbe",
    "cf.": "confer (voir)",
    "c.-à-d.": "c'est-à-dire",
    "etc.": "et cetera (et ainsi de suite)",
    "ex.": "exemple",
    "i.e.": "id est (c'est-à-dire)",
    "journ.": "journal",
    "jr": "jour",
    "l.c.": "lettre capitale",
    "lit.": "littéralement",
    "loc. cit.": "loco citato (au même endroit)",
    "m.": "masculin",
    "masc.": "masculin",
    "min.": "minute",
    "mois.": "mois",
    "n.": "nom",
    "n.d.a.": "note de l'auteur",
    "n.d.l.r.": "note de la rédaction",
    "n.m.": "nom masculin",
    "n.f.": "nom féminin",
    "p.": "page",
    "pl.": "pluriel",
    "s.d.": "sans date",
    "s.n.": "sans nom (de l'éditeur)",
    "s.v.p.": "s'il vous plaît",
    "sig.": "signature",
    "sing.": "singulier",
    "sq.": "suivant (paragraphe)",
    "ss.": "suivants (paragraphes)",
    "s.v.": "sub verbo (sous le mot)",
    "tél.": "téléphone",
    "vol.": "volume",
    "vs": "versus (contre)"
}

In [128]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'URL',text)


# Remove HTML beacon
def remove_HTML(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# Remove non printable characters
def remove_not_ASCII(text):
    text = ''.join([word for word in text if word in string.printable])
    return text

# Change an abbreviation by its true meaning
def word_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

# Replace all abbreviations
def replace_abbrev(text):
    string = ""
    for word in text.split():
        string += word_abbrev(word) + " "        
    return string

# Remove @ and mention, replace by USER
def remove_mention(text):
    at=re.compile(r'@\S+')
    return at.sub(r'USER',text)
                     

# Remove numbers, replace it by NUMBER
def remove_number(text):
    num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
    return num.sub(r'NUMBER', text)



# Replace some others smileys with SADFACE
def transcription_sad(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[(\\/]')
    return smiley.sub(r'SADFACE', text)


# Replace some smileys with SMILE
def transcription_smile(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[)dDp]')
    #smiley = re.compile(r'#{eyes}#{nose}[)d]+|[)d]+#{nose}#{eyes}/i')
    return smiley.sub(r'SMILE', text)


# Replace <3 with HEART
def transcription_heart(text):
    heart = re.compile(r'<3')
    return heart.sub(r'HEART', text)


# Factorize elongated words, add ELONG
def remove_elongated_words(text):
    rep = re.compile(r'\b(\S*?)([a-z])\2{2,}\b')
    return rep.sub(r'\1\2 ELONG', text)


# Factorize repeated punctuation, add REPEAT
def remove_repeat_punct(text):
    rep = re.compile(r'([!?.]){2,}')
    return rep.sub(r'\1 REPEAT', text)



# Remove all punctuations
def remove_all_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)


# Remove punctuations
def remove_punct(text):
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" 
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    text = text.replace('...', ' ... ')
    if '...' not in text:
        text = text.replace('..', ' ... ')   
    return text


# Remove all english stopwords
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in stopwords.words("english")])
    return text

def stemming(text):
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def lemmatization(text):
    lm= WordNetLemmatizer()
    text = ' '.join([lm.lemmatize(word, pos='v') for word in text.split()])
    return text

In [129]:
def clean_text(text):
    
    # Remove non text
    text = remove_URL(text)
    text = remove_HTML(text)
    text = remove_not_ASCII(text)
    
    # Lower text, replace abbreviations
    text = replace_abbrev(text)  
    text = remove_mention(text)
    text = remove_number(text)
    
    # Remove  smileys

    text = transcription_sad(text)
    text = transcription_smile(text)
    text = transcription_heart(text)
    
    # Remove repeated puntuations / words
    text = remove_elongated_words(text)
    text = remove_repeat_punct(text)

    text = remove_all_punct(text)
    text = remove_punct(text)
    text = lemmatization(text)
    text = text.lower()

    return text


In [147]:
data['emailclean'] = data['text'].apply(clean_text)

In [148]:
embed = hub.load("C:/Users/PC/OneDrive - Université du Québec en Outaouais/Documents/categorization/savedmodel")

In [149]:
sentences = data["emailclean"].apply(lambda x: x.lower()).tolist()
# generating training data matrix
embed_matrix = []
for sent in sentences:
    embed_matrix.append(np.array(embed([sent])[0]).tolist())

In [150]:
# Transforming catagories label to 
labels = data['category'].astype(str)

lb = LabelBinarizer().fit(list(set(labels.tolist())))

In [151]:
# setting up train data and labels (processed)
X, Y = np.array(embed_matrix), lb.transform(data['category'].tolist())

In [152]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,stratify=Y, random_state=42)

In [162]:
import tensorflow as tf
# Model Architecture
ann = tf.keras.models.Sequential()
ann.add(tf.keras.Input(shape=512))
ann.add(tf.keras.layers.Dense(units=512, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.5))
ann.add(tf.keras.layers.Dense(units=256, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.5))
ann.add(tf.keras.layers.Dense(units=128, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.5))
ann.add(tf.keras.layers.Dense(units=64, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.5))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=5, activation='softmax'))
# Compile the model
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
ann.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy']) 

# Set up early stopping callback function
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [163]:
# Train the model
history = ann.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[es_callback])

Epoch 1/10
184/184 [==============================] - 6s 25ms/step - loss: 1.4889 - categorical_accuracy: 0.3382 - val_loss: 1.2532 - val_categorical_accuracy: 0.5026
Epoch 2/10
184/184 [==============================] - 4s 23ms/step - loss: 1.2213 - categorical_accuracy: 0.5039 - val_loss: 1.0896 - val_categorical_accuracy: 0.5436
Epoch 3/10
184/184 [==============================] - 4s 24ms/step - loss: 1.0857 - categorical_accuracy: 0.5505 - val_loss: 1.0268 - val_categorical_accuracy: 0.5699
Epoch 4/10
184/184 [==============================] - 4s 24ms/step - loss: 1.0124 - categorical_accuracy: 0.5766 - val_loss: 0.9390 - val_categorical_accuracy: 0.6093
Epoch 5/10
184/184 [==============================] - 4s 24ms/step - loss: 0.9558 - categorical_accuracy: 0.6074 - val_loss: 0.9539 - val_categorical_accuracy: 0.6157
Epoch 6/10
184/184 [==============================] - 4s 23ms/step - loss: 0.8941 - categorical_accuracy: 0.6284 - val_loss: 0.9331 - val_categorical_accuracy: 0.612

In [107]:
# Importer les modules nécessaires
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

# Faire des prédictions sur les données de test
y_pred = ann.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Convertir y_test en une liste de labels de classe
y_test_class = np.argmax(y_test, axis=1)

# Calculer la matrice de confusion et afficher le rapport de classification
cm = confusion_matrix(y_test_class, y_pred)
cr = classification_report(y_test_class, y_pred)

# Calculer la précision, le rappel et le score F1 pour chaque classe
precision = precision_score(y_test_class, y_pred, average=None)
recall = recall_score(y_test_class, y_pred, average=None)
f1 = f1_score(y_test_class, y_pred, average=None)

# Calculer l'exactitude
accuracy = accuracy_score(y_test_class, y_pred)

print('Confusion Matrix:\n', cm)
print('Classification Report:\n', cr)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('Accuracy:', accuracy)


21/21 [==============================] - 0s 7ms/step
Confusion Matrix:
 [[151   0   1   0   1]
 [  0 116   0   0   0]
 [  1   0 120   3   1]
 [  0   0   0 154   0]
 [  1   0   0   0 119]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       153
           1       1.00      1.00      1.00       116
           2       0.99      0.96      0.98       125
           3       0.98      1.00      0.99       154
           4       0.98      0.99      0.99       120

    accuracy                           0.99       668
   macro avg       0.99      0.99      0.99       668
weighted avg       0.99      0.99      0.99       668

Precision: [0.9869281  1.         0.99173554 0.98089172 0.98347107]
Recall: [0.9869281  1.         0.96       1.         0.99166667]
F1 Score: [0.9869281  1.         0.97560976 0.9903537  0.98755187]
Accuracy: 0.9880239520958084


In [165]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"
import tensorflow_text as text
import tensorflow.compat.v1 as tf

bert_preprocess = hub.KerasLayer("C:/Users/PC/OneDrive - Université du Québec en Outaouais/Documents/categorization/bertprocess")
bert_encoder = hub.KerasLayer("C:/Users/PC/OneDrive - Université du Québec en Outaouais/Documents/categorization/EncoderBERT")

In [166]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [173]:
df_business = data[data['category'] == 'business']
df_politics = data[data['category'] == 'politics']
df_enter = data[data['category'] == 'entertainment']
df_sport = data[data['category'] == 'sport']
df_tech = data[data['category'] == 'tech']

In [167]:
from sklearn.model_selection import train_test_split
# Diviser les données en train et test
train_texts, test_texts, train_labels, test_labels = train_test_split(data['emailclean'],data['label'], test_size=0.3, random_state=42)

In [168]:
train_labelss = tf.convert_to_tensor(train_labels, dtype=tf.float32)

In [179]:
# Entraînement du modèle de classification

from keras.models import Sequential

from keras.layers import Dense, Dropout, Embedding, LSTM

from keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences

# Créer des modèles de classification de spam pour chaque catégorie

spam_models = {}

for categorie in ['business', 'politics', 'tech', 'sport', 'entertainment']:

    # Sélectionner les données pour cette catégorie

    df_categorie = data[data['category'] == categorie]

    train_texts_categorie = df_categorie['emailclean']

    train_labels_categorie = df_categorie['label']

    # Diviser les données en train et validation

    train_texts_categorie, val_texts_categorie, train_labels_categorie, val_labels_categorie = train_test_split(train_texts_categorie, train_labels_categorie, test_size=0.2, random_state=42)

    # Convertir les labels en tenseurs

    train_labelss_categorie = tf.convert_to_tensor(train_labels_categorie, dtype=tf.float32)

    val_labelss_categorie = tf.convert_to_tensor(val_labels_categorie, dtype=tf.float32)

    # Encoder les phrases avec BERT

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

    embedding = tf.keras.layers.Lambda(get_sentence_embeding, name='embedding')(text_input)

    dense = Dense(256, activation='relu', name='dense')(embedding)

    dropout = Dropout(0.1, name='dropout')(dense)

    output = Dense(1, activation='sigmoid', name='output')(dropout)

    spam_model = tf.keras.Model(inputs=[text_input], outputs=[output])

    # Compilation et entraînement du modèle

    spam_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Utilisation de la méthode cache pour mettre en cache les données en mémoire pour accélérer l'entraînement du modèle

    train_data = tf.data.Dataset.from_tensor_slices((train_texts_categorie.astype(str), train_labelss_categorie)).cache().batch(32)

    val_data = tf.data.Dataset.from_tensor_slices((val_texts_categorie.astype(str), val_labelss_categorie)).cache().batch(32)

    history = spam_model.fit(train_data, epochs=10, batch_size=32, validation_data=val_data)

    # Enregistrer le modèle pour cette catégorie

    spam_models[categorie] = spam_model


The following Variables were used a Lambda layer's call (embedding), but
are not present in its tracked objects:
  <tf.Variable 'pooler_transform/bias:0' shape=(768,) dtype=float32>
  <tf.Variable 'pooler_transform/kernel:0' shape=(768, 768) dtype=float32>
  <tf.Variable 'transformer/layer_11/output_layer_norm/beta:0' shape=(768,) dtype=float32>
  <tf.Variable 'transformer/layer_11/output_layer_norm/gamma:0' shape=(768,) dtype=float32>
  <tf.Variable 'transformer/layer_11/output/bias:0' shape=(768,) dtype=float32>
  <tf.Variable 'transformer/layer_11/output/kernel:0' shape=(3072, 768) dtype=float32>
  <tf.Variable 'transformer/layer_11/intermediate/bias:0' shape=(3072,) dtype=float32>
  <tf.Variable 'transformer/layer_11/intermediate/kernel:0' shape=(768, 3072) dtype=float32>
  <tf.Variable 'transformer/layer_11/self_attention_layer_norm/beta:0' shape=(768,) dtype=float32>
  <tf.Variable 'transformer/layer_11/self_attention_layer_norm/gamma:0' shape=(768,) dtype=float32>
  <tf.Variable

Epoch 1/10
61/61 [==============================] - 544s 9s/step - loss: 0.6553 - accuracy: 0.6419 - val_loss: 0.5783 - val_accuracy: 0.6867
Epoch 2/10
61/61 [==============================] - 533s 9s/step - loss: 0.5231 - accuracy: 0.7504 - val_loss: 0.5194 - val_accuracy: 0.7344
Epoch 3/10
61/61 [==============================] - 534s 9s/step - loss: 0.4377 - accuracy: 0.8220 - val_loss: 0.4348 - val_accuracy: 0.8050
Epoch 4/10
61/61 [==============================] - 535s 9s/step - loss: 0.3972 - accuracy: 0.8402 - val_loss: 0.3890 - val_accuracy: 0.8402
Epoch 5/10
61/61 [==============================] - 530s 9s/step - loss: 0.3633 - accuracy: 0.8588 - val_loss: 0.3616 - val_accuracy: 0.8651
Epoch 6/10
61/61 [==============================] - 533s 9s/step - loss: 0.3412 - accuracy: 0.8723 - val_loss: 0.3460 - val_accuracy: 0.8651
Epoch 7/10
61/61 [==============================] - 531s 9s/step - loss: 0.3436 - accuracy: 0.8682 - val_loss: 0.3518 - val_accuracy: 0.8672
Epoch 8/10
61

Epoch 1/10
55/55 [==============================] - 480s 9s/step - loss: 0.7332 - accuracy: 0.5453 - val_loss: 0.6629 - val_accuracy: 0.5553
Epoch 2/10
55/55 [==============================] - 471s 9s/step - loss: 0.6567 - accuracy: 0.6151 - val_loss: 0.6380 - val_accuracy: 0.6198
Epoch 3/10
55/55 [==============================] - 471s 9s/step - loss: 0.6351 - accuracy: 0.6451 - val_loss: 0.6213 - val_accuracy: 0.6359
Epoch 4/10
55/55 [==============================] - 470s 9s/step - loss: 0.6177 - accuracy: 0.6595 - val_loss: 0.6058 - val_accuracy: 0.6498
Epoch 5/10
55/55 [==============================] - 470s 9s/step - loss: 0.5999 - accuracy: 0.6780 - val_loss: 0.5931 - val_accuracy: 0.6682
Epoch 6/10
55/55 [==============================] - 470s 9s/step - loss: 0.5800 - accuracy: 0.7017 - val_loss: 0.6006 - val_accuracy: 0.6429
Epoch 7/10
55/55 [==============================] - 472s 9s/step - loss: 0.5595 - accuracy: 0.7253 - val_loss: 0.5797 - val_accuracy: 0.6659
Epoch 8/10
55

Epoch 1/10
39/39 [==============================] - 446s 11s/step - loss: 0.6236 - accuracy: 0.7030 - val_loss: 0.5212 - val_accuracy: 0.7276
Epoch 2/10
39/39 [==============================] - 434s 11s/step - loss: 0.4914 - accuracy: 0.7689 - val_loss: 0.4636 - val_accuracy: 0.7564
Epoch 3/10
39/39 [==============================] - 435s 11s/step - loss: 0.4451 - accuracy: 0.7986 - val_loss: 0.4121 - val_accuracy: 0.8173
Epoch 4/10
39/39 [==============================] - 437s 11s/step - loss: 0.4053 - accuracy: 0.8194 - val_loss: 0.3795 - val_accuracy: 0.8333
Epoch 5/10
39/39 [==============================] - 434s 11s/step - loss: 0.3729 - accuracy: 0.8395 - val_loss: 0.3512 - val_accuracy: 0.8429
Epoch 6/10
39/39 [==============================] - 432s 11s/step - loss: 0.3488 - accuracy: 0.8523 - val_loss: 0.3251 - val_accuracy: 0.8718
Epoch 7/10
39/39 [==============================] - 444s 11s/step - loss: 0.3247 - accuracy: 0.8620 - val_loss: 0.3087 - val_accuracy: 0.8814
Epoch 

Epoch 1/10
34/34 [==============================] - 366s 11s/step - loss: 0.7277 - accuracy: 0.5602 - val_loss: 0.6588 - val_accuracy: 0.5955
Epoch 2/10
34/34 [==============================] - 360s 11s/step - loss: 0.6677 - accuracy: 0.5827 - val_loss: 0.6390 - val_accuracy: 0.6554
Epoch 3/10
34/34 [==============================] - 361s 11s/step - loss: 0.6501 - accuracy: 0.6043 - val_loss: 0.6164 - val_accuracy: 0.7154
Epoch 4/10
34/34 [==============================] - 360s 11s/step - loss: 0.6243 - accuracy: 0.6635 - val_loss: 0.6072 - val_accuracy: 0.7079
Epoch 5/10
34/34 [==============================] - 357s 11s/step - loss: 0.6095 - accuracy: 0.6795 - val_loss: 0.5978 - val_accuracy: 0.7116
Epoch 6/10
34/34 [==============================] - 359s 11s/step - loss: 0.5982 - accuracy: 0.6964 - val_loss: 0.6002 - val_accuracy: 0.7041
Epoch 7/10
34/34 [==============================] - 359s 11s/step - loss: 0.5802 - accuracy: 0.7152 - val_loss: 0.5841 - val_accuracy: 0.7004
Epoch 

Epoch 1/10
23/23 [==============================] - 246s 11s/step - loss: 0.6560 - accuracy: 0.6708 - val_loss: 0.5781 - val_accuracy: 0.6961
Epoch 2/10
23/23 [==============================] - 243s 11s/step - loss: 0.5413 - accuracy: 0.7248 - val_loss: 0.5478 - val_accuracy: 0.7182
Epoch 3/10
23/23 [==============================] - 242s 11s/step - loss: 0.5203 - accuracy: 0.7455 - val_loss: 0.6362 - val_accuracy: 0.6961
Epoch 4/10
23/23 [==============================] - 243s 11s/step - loss: 0.4897 - accuracy: 0.7552 - val_loss: 0.5724 - val_accuracy: 0.7017
Epoch 5/10
23/23 [==============================] - 244s 11s/step - loss: 0.4631 - accuracy: 0.7746 - val_loss: 0.5401 - val_accuracy: 0.7072
Epoch 6/10
23/23 [==============================] - 249s 11s/step - loss: 0.4514 - accuracy: 0.7828 - val_loss: 0.5710 - val_accuracy: 0.7127
Epoch 7/10
23/23 [==============================] - 239s 10s/step - loss: 0.4235 - accuracy: 0.8064 - val_loss: 0.4938 - val_accuracy: 0.7182
Epoch 

In [184]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Évaluation des performances des modèles sur les données de test

for i, (categorie, model) in enumerate(spam_models.items()):

    print(f"Performance du modèle de classification {i+1} : {categorie}")

    val_loss, val_accuracy = model.evaluate(val_texts_categorie, val_labels_categorie, verbose=0)

    print(f"Perte : {val_loss:.4f}")

    print(f"Précision : {val_accuracy:.4f}")

    # Faire des prédictions sur les données de validation
    y_val_true = val_labelss_categorie.numpy().astype(int)
    y_val_pred = np.round(model.predict(val_data)).astype(int)

    # Calculer la matrice de confusion et le rapport de classification
    cm = confusion_matrix(y_val_true, y_val_pred)
    cr = classification_report(y_val_true, y_val_pred)

    # Calculer l'exactitude
    acc = accuracy_score(y_val_true, y_val_pred)

    # Afficher les résultats
    print('Matrice de confusion:\n', cm)
    print('Rapport de classification:\n', cr)
    print('Exactitude:', acc)


Performance du modèle de classification 1 : business
Perte : 0.2448
Précision : 0.9171
6/6 [==============================] - 50s 8s/step
Matrice de confusion:
 [[119   4]
 [ 11  47]]
Rapport de classification:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94       123
           1       0.92      0.81      0.86        58

    accuracy                           0.92       181
   macro avg       0.92      0.89      0.90       181
weighted avg       0.92      0.92      0.92       181

Exactitude: 0.9171270718232044
Performance du modèle de classification 2 : politics
Perte : 0.4555
Précision : 0.8398
6/6 [==============================] - 58s 10s/step
Matrice de confusion:
 [[116   7]
 [ 22  36]]
Rapport de classification:
               precision    recall  f1-score   support

           0       0.84      0.94      0.89       123
           1       0.84      0.62      0.71        58

    accuracy                           0.84       181

In [ ]:
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

for history_dict in list_of_histories:
    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.subplot(2, 1, 1)
    # r is for "solid red line"
    plt.plot(epochs, loss, 'r', label='Training loss')
    # b is for "solid blue line"
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    # plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(epochs, acc, 'r', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

plt.show()

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
dfnew = pd.read_csv("bbc-text.csv")

In [ ]:
dfnew['email']=dfnew['text']

In [81]:
Xnew=dfnew["email"]

In [64]:
print(Xnew[1])

worldcom boss  left books alone  former worldcom boss bernie ebbers  who is accused of overseeing an $11bn (£5.8bn) fraud  never made accounting decisions  a witness has told jurors.  david myers made the comments under questioning by defence lawyers who have been arguing that mr ebbers was not responsible for worldcom s problems. the phone company collapsed in 2002 and prosecutors claim that losses were hidden to protect the firm s shares. mr myers has already pleaded guilty to fraud and is assisting prosecutors.  on monday  defence lawyer reid weingarten tried to distance his client from the allegations. during cross examination  he asked mr myers if he ever knew mr ebbers  make an accounting decision  .  not that i am aware of   mr myers replied.  did you ever know mr ebbers to make an accounting entry into worldcom books   mr weingarten pressed.  no   replied the witness. mr myers has admitted that he ordered false accounting entries at the request of former worldcom chief financia

In [66]:
Znew=dfnew['category']

In [69]:
len(Xnew)

2225

In [90]:
ar = np.array([Xnew[1]])
df10= pd.DataFrame(ar, columns=['text'])

In [91]:
print(df10)

                                                text
0  worldcom boss  left books alone  former worldc...


In [93]:
y_predicted_bus = modelbusiness.predict(df10) 
y_predicted_bus = y_predicted_bus.flatten() 
#spam.append(np.where(y_predicted_bus > 0.5, 1, 0))

1/1 [==============================] - 0s 300ms/step


In [68]:
Znew.shape

(2225,)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='email')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [87]:
y_predicted_bus = modelbusiness.predict(df6['text']) 
y_predicted_bus = y_predicted_bus.flatten() 
print(np.where(y_predicted_bus > 0.5, 1, 0))

16/16 [==============================] - 103s 6s/step
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [95]:
spam = []

for i in range(len(Xnew)): 
    if Znew[i]=='business' : 
        ar = np.array([Xnew[i]])
        df10= pd.DataFrame(ar, columns=['text'])     
        y_predicted_bus = modelbusiness.predict(df10) 
        y_predicted_bus = y_predicted_bus.flatten() 
        spam.append(np.where(y_predicted_bus > 0.5, 1, 0)) 
    elif Znew[i]=='tech' :
        ar = np.array([Xnew[i]])
        df10= pd.DataFrame(ar, columns=['text'])
        y_predicted_tech = modelTech.predict(df10) 
        y_predicted_tech = y_predicted_tech.flatten() 
        spam.append(np.where(y_predicted_tech > 0.5, 1, 0)) 
    elif Znew[i]=='entertainement': 
        ar = np.array([Xnew[i]])
        df10= pd.DataFrame(ar, columns=['text'])
        y_predicted_Entert = modelEntertainment.predict(df10) 
        y_predicted_Entert = y_predicted_Entert.flatten() 
        spam.append(np.where(y_predicted_Entert > 0.5, 1, 0)) 
    elif Znew[i]=='sport' : 
        ar = np.array([Xnew[i]])
        df10= pd.DataFrame(ar, columns=['text'])
        y_predicted_spo = modelsport.predict(df10) 
        y_predicted_spo = y_predicted_spo.flatten() 
        spam.append(np.where(y_predicted_spo > 0.5, 1, 0)) 
    else : 
        ar = np.array([Xnew[i]])
        df10= pd.DataFrame(ar, columns=['text'])
        y_predicted_polit = modelpolitics.predict(df10) 
        y_predicted_polit= y_predicted_polit.flatten() 
        spam.append(np.where(y_predicted_polit > 0.5, 1, 0))


1/1 [==============================] - 0s 246ms/step


1/1 [==============================] - 0s 316ms/step


1/1 [==============================] - 0s 282ms/step


1/1 [==============================] - 0s 272ms/step


1/1 [==============================] - 0s 291ms/step


1/1 [==============================] - 0s 286ms/step


1/1 [==============================] - 0s 408ms/step


1/1 [==============================] - 0s 319ms/step


In [102]:
dfnew['spam']=pd.DataFrame(spam)

In [108]:
R=dfnew['spam']

In [104]:
New_df_emails = dfnew.to_csv('New_df_emails.csv', index = True)